![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/Clinical_Deidentification_Multilingual.ipynb)

## **Deidentification Model for Different Languages**

## **1. Colab Setup**

In [1]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
import os
os.environ.update(license_keys)

Saving 3.4.2.spark_nlp_for_healthcare1.json to 3.4.2.spark_nlp_for_healthcare1.json


In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [4]:
import json
import os
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

from sparknlp.util import *
from sparknlp.pretrained import ResourceDownloader
from pyspark.sql import functions as F

import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

import string
import numpy as np

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(secret = SECRET, params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

Spark NLP Version : 3.4.2
Spark NLP_JSL Version : 3.4.2


In [5]:
from sparknlp.pretrained import PretrainedPipeline
import pandas as pd

In [6]:
spark

## **2. Sample Examples for all of the 10 Different languages**

In [7]:
text_english = ["""Record date: 2069-04-07
Mr. Villegas is seen today.  I have not seen him since November. 
About three weeks ago he stopped his Prednisone on his own because
he was gaining weight.  He does feel that his shoulders are
definitely improved.
On examination today, BP 120/80.  His joint examination is much
improved with better ROM of the shoulders and no peripheral joint
synovitis.  
Clinical Impression:
1:  Inflammatory arthritis - possibly RA - with response noted to
Hydroxychloroquine along with Prednisone.  He has stopped the
Prednisone, and I would not restart it yet.  
2:  New onset of symptoms suspicious for right-sided carotid
disease.  Will arrange for carotid ultrasound studies.  Patient
advised to call me if he develops any worsening symptoms.
Xzavian G. Tavares, M.D.
XGT:holmes
DD: 04/07/69
DT: 04/15/69
DV: 04/07/69""",
"""Record date: 2080-02-18
SDU JAR Admission Note
Name: 	 Yosef Villegas
MR:	8249813
DOA:  	2/17/80
PCP:  Gilbert Perez
Attending: YBARRA
CODE: FULL
HPI:  70 yo M with NIDDM admitted for cath after positive MIBI. Pt has had increasing CP and SOB on exertion for past two months, seen at PCP on 2/03 with EKG showing RBBB. Pt was scheduled for stress MIBI today to eval for cardiac ischemia. MIBI was reportedly stopped 2/2 to chest pain, though pt denies having pain during procedure. MIBI was read as positive for moderate to severe inferior ischemia and pt was referred to the ED for admission and catheterization.
PMH: 
Seronegative Rheumatoid Arthritis 
HTN
Right Carotid Artery Dz 
Obesity 
Hyperlipidemia 
Diabetes mellitus type 2
Home Medications: 
Acetylsalicylic Acid (Aspirin)  PO 81 MG QD
Xanax (Alprazolam)  PO  BID
Atenolol  PO 50 MG QD
Enalapril Maleate  PO 10 MG QD
Hctz (Hydrochlorothiazide)  PO  QD
Metformin  PO 850 MG BID
Allergies: NKDA
Social History:  Smokes 2 packs per month, drinks 1 case of beer per month.
Mike Ivan, MD, EHMS pager 84710
""",
"""Record date : 2093-01-13, David Hale, M.D. is manager, 
Name: Hendrickson, Ora MR. # 7194334 Date: 01-13-1993 PCP: Oliveira.
Record date: 2079-11-09. Cocke County Baptist Hospital. 0295 Keats Street.
This 17-yr-old male, presented with chest heaviness that started during a pick-up basketball game. His past medical history was unremarkable. He denied prior cardiac symptoms and suffered no chest trauma during the game. His father had suffered an acute myocardial infarction at age 38. The patient was a nonsmoker, did not drink alcohol, and denied recreational drug use. He swallowed a tablet of aspirin before coming to the emergency room. His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. Physical examination revealed no stigmata of Marfan syndrome. The rest of his physical examination was normal.""",
"""Patient AIQING, 25 month years-old , born in Beijing, was transfered to the The Johns Hopkins Hospital. Phone number: (541) 754-3010. MSW 100009632582 for his colonic polyps.  He wants to know the results from them.  He is not taking hydrochlorothiazide and is curious about his blood
pressure. He said he has cut his alcohol back to 6 pack once a week.  He has cut back his cigarettes to one time per week.
P:   Follow up with Dr. Hobbs in 3 months.
Gilbert P. Perez, M.D.""",
"""A . Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR . # 7194334 Date : 01-13-1993 PCP : Oliveira , 25 month years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street"""
      ]

text_german = ["""Zusammenfassung : Michael Berger wird am Morgen des 12 Dezember 2018 ins St.Elisabeth Krankenhaus eingeliefert. 
Herr Michael Berger ist 76 Jahre alt und hat zu viel Wasser in den Beinen.Persönliche Daten :ID-Nummer: T0110053F Platte A-BC124 Kontonummer: DE89370400440532013000 SSN : 13110587M565 Lizenznummer: B072RRE2I55 Adresse : St.Johann-Straße 13 19300
""",
"""Herr Reus - Telefon: 0604876475 - Beruf: Koch an der Friedrich-Alexander Universität. Er berichtet über Krankenhausaufenthalte im Klinikum Nürnberg-Süd, und die Akte zeigt weitere Krankenhausaufenthalte, darunter Kreiskrankenhaus im Jahr 2001, Sankt Antonius Krankenhaus am 28 Juli 2000, Klinikum Ludwigsburg im Jahr 05/1966.""",
"""Koronare Herzkrankheit : s/p ant SEMI + Stent LAD Aug/1963 , Dr . Lars Meister , ETT Fallingbostel 24.02.2009 - neg . Scan auf Ischämie .""",
"""Dr. Hans-Wolfgang Weihmann - RM57, Städt Klinikum Dresden-Friedrichstadt, Friedrichstraße 41, Dresden""",
"""Er arbeitete bis 24.08.1940 - Gärtner bei Planten un Blomen in Hamburg, verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. Klein im September."""]

text_spanish = ["""Nombre: Danilo.
Apellidos: Ramos Portillo.
NHC: 4376372.
NASS: 35 61051794 56.
Domicilio: Calle Fernando Higueras, 27, 7B.
Localidad/ Provincia: Madrid.
CP: Madrid 2805528055.
Datos asistenciales.
Fecha de nacimiento: 15/07/1979.
País: España.
Edad: 38 años Sexo: H.
Fecha de Ingreso: 04/03/2014.
Episodio: 82034567902.
Médico: Fabio Jiménez Borrero NºCol: 28 28 57986.
Informe clínico del paciente.
Historia Actual: Paciente varón de 38 años, valorado en consulta por coxalgia derecha de año y medio de evolución, de características mecánicas. Refiere dolor con práctica de ejercicio físico y leve limitación de la movilidad en ciertas posturas. Mecánico de profesión, en su tiempo libre realizaba bicicleta (40 km diarios en fines de semana), carrera 3 veces por semana durante 50 minutos y marcha de una hora los días que no realizaba carrera. El dolor se ha ido incrementando progresivamente, realizando, en la actualidad, solamente marcha.
Exploración física: A la exploración física destaca un dolor a la palpación en tercio medio de la región inguinal derecha con dolor a la flexión, aducción y rotaciones sobre todo interna (maniobra de choque positiva). El balance articular de caderas es de flexión 110o bilateral, rotación interna de 10o derecha, 40o izquierda y rotación externa de 30o derecha, 40o izquierda. El balance muscular en la escala de Daniels es de 5/5 global bilateral. No presenta dolor a la palpación de puntos ciáticos y las maniobras de Valleix, Fabere, Lasegue y Bragard son negativas. Tampoco presenta dolor a la palpación en la cintilla iliotibial y el test de Ober es negativo. En la exploración de la marcha observamos ausencia de claudicación y el apoyo monopodal alterno derecho-izquierdo es estable, con Trendelenburg negativo. Presenta dolor para la marcha con flexión forzada de cadera y rodilla.
Pruebas complementarias: La radiología simple anteroposterior (AP) presentaba una deformidad en forma de joroba en región cérvico-cefálica femoral derecha. La analítica solicitada con hemograma, bioquímica, hormonas tiroideas, anticuerpos y reactantes de fase aguda era normal.
Ante estos hallazgos se solicita resonancia magnética de cadera en la que se visualiza leve alteración subcondral, con edema óseo en cadera derecha, con irregularidad cortical y leve sinovitis en interlínea articular, con distensión de la bursa ilio-psoas. La cabeza femoral presenta un pequeño islote óseo y el ángulo alfa es mayor de 50o. La artrografía muestra un pinzamiento de la interlinea articular antero-superior y posteroinferior, un incipiente collar osteofitario femoral con pequeñas lesiones subcondrales antero-superiores de cabeza femoral y pequeños focos de edema en acetábulo anterosuperior. No se visualiza necrosis avascular ni lesiones en el labrum. También se visualiza una lesión quística yuxtaarticular en relación con trocánter mayor.
Evolución: Con el diagnóstico de síndrome de pinzamiento femoroacetabular tipo LEVA (también conocido por CAM) se le prescribe al paciente tratamiento con magnetoterapia (15 sesiones), para mejorar la clínica del dolor y el trofismo óseo.
Se remite a unidad de artroscopia del servicio de traumatología para valoración de cirugía artroscópica.
Diagnóstico Principal: síndrome de pinzamiento femoroacetabular tipo LEVA
Tratamiento: Se pauta tratamiento farmacológico con asociación de sulfato de glucosamina-condroitin sulfato oral 400 mg cada 12 horas durante 3 meses y antiinflamatorios no esteroideos a demanda si presenta dolor. Se le indica reposo deportivo para evitar choque en la articulación, con actividad física activa controlada, recomendando no sobrepasar los 90 grados de flexión de la cadera y evitar las rotaciones. Acudir a cita para revisión una vez terminados los sulfatos de glucosamina-condroitin para ver la mejoría clínica, deberá llamar al numero de atención y citas previas de hospital 914 90 66 60
Remitido por: Dr. Fabio Jiménez Borrero. Hospital Universitario La Paz. Paseo de la Castellana, 261, 28046 Madrid. Teléfono: 917 27 70 00. Correo electrónico: fabio_jimenezb@salud.madrid.org 
""",
"""Nombre: David.
Apellidos: Flores Blanco.
NHC: 8972604.
Domicilio:  Calle Puerto Rico, 17, 3 C.
Localidad/ Provincia: Albacete.
CP: 02541.
Datos asistenciales.
Fecha de nacimiento: 20/10/2003.
País: España.
Edad: 13 años Sexo: H.
Fecha de Ingreso: 04/01/2017.
Médico: María Flores Copete NºCol: 02 02 12630.
Informe clínico del paciente: Niño de 13 años, con antecedentes personales de síndrome de Asperger, trastorno de déficit de atención e hiperactividad e hipotiroidismo, en tratamiento con metilfenidato, risperidona y levotiroxina. Fue enviado a la consulta de Dermatología por presentar una lesión en el miembro inferior derecho, de crecimiento progresivo, desde hacía un año. Era asintomática, sin presentar ninguna clínica como dolor o prurito. No había recibido ningún tratamiento, ni refería ningún factor desencadenante. En la exploración se observó una placa, indurada a la palpación, con un área discrómica central y un halo violáceo, de distribución lineal en el tercio medio-distal en la zona externa del muslo derecho. No presentaba otras lesiones cutáneas, ni clínica sistémica acompañante.
Padre con rasgos de Asperger no diagnosticado.
Se solicitó una analítica que incluía hemograma, bioquímica, anticuerpos antinucleares (ANA) y serologías, examen de orina, así como biopsia de la lesión. Los resultados fueron normales, con ANA, anti-Sm, anti-RNP, anti-SSA, anti-SSB, anti-Jo1 y anti-Scl70 negativos. La serología para Borrelia burgdorferi fue negativa y el análisis de orina no mostró anomalías. La histopatología fue compatible con el diagnóstico de morfea.
Ante una morfea lineal en la infancia sin afectación articular ni deformidad estética, se decidió iniciar tratamiento corticoideo tópico con propionato de fluticasona, una aplicación al día durante 2 meses, seguido de calcipotriol tópico, también un aplicación diaria. Tras casi un año de seguimiento, las lesiones se han estabilizado.
Remitido por: Dra. María Flores Copete. Centro de Salud Zona 4. C/ Seminario, 4. C.P. 02006. Albacete. España. Correo electrónico: florescopetemaria@gmail.com
""",
"""Nombre: Alexia.
Apellidos: Hernandez Peña.
NHC: 5347636/301.
Domicilio: Av. Sta. María 281, Recoleta, Región Metropolitana.
Localidad/ Provincia: Santiago.
CP: 8420000.
Datos asistenciales.
Fecha de nacimiento: 21/06/1971.
País: Chile.
Edad: 44 años Sexo: Mujer.
Fecha de Ingreso: 09/08/2015.
Médico: Julio Villanueva Maffei.
Motivo de ingreso: Paciente mujer, 44 años, derivado a nuestra unidad por aumento de volumen en la región cigomática y limitación progresiva de la apertura bucal de 10 meses de evolución. La historia médica no era relevante. 
Exploración: El examen físico extraoral muestra asimetría facial por aumento de volumen en la mejilla izquierda, de limites difusos, consistencia ósea, indoloro, con piel de aspecto normal. La apertura bucal era 30 mm. La articulación temporomandibular (ATM) era normal a la palpación, sin ruidos o dolor en apertura. En la ortopantomografía observamos una apófisis coronoides izquierda de mayor tamaño. La hipótesis diagnóstica fue tumor coronoídeo.
Pruebas complementarias y evolución:  Se realizó una incisión por sobre el borde anterior de la rama mandibular, desinsertando todas las inserciones del músculo temporal. Alrededor del aumento de volumen se encontró una pseudocápsula fibrosa, que fue liberada. Se efectuó la coronoidectomía. La apertura bucal se recuperó inmediatamente hasta 43 mm. El postoperatorio se desarrolló sin complicaciones, y la paciente fue dada de alta a las 48 horas.
El estudio histopatológico informó la presencia fibras, hueso de neoformación y tejido cartilaginoso hialino. El diagnóstico fue osteocondroma. Los controles posteriores se desarrollaron sin problemas. El alta definitiva fue dada a los 10 meses, con remodelación casi completa de la deformidad de la mejilla.
Remitido por: Dr. Julio Villanueva Maffei. Santa María 571, Recoleta Chile E-mail: javm@vtr.net""",
"""Nombre: Vicente.
Apellidos: Manrique Bonilla.
NHC: 7831956.
NASS:37 39843608 49.
Domicilio: Calla la Pedrera, 24, 7D.
Localidad/ Provincia: Madrid.
CP: 28035.
Datos asistenciales.
Fecha de nacimiento: 27/11/1964.
País: España.
Edad: 52 años Sexo: H.
Fecha de Ingreso: 21/05/2016.
Médico: Verónica Blanco Gil  NºCol: 28 28.
Informe clínico del paciente: Hombre de 52 años, de raza blanca, que consulta en lel servicio de odontologia por dolor producido por una lesión exofítica en mucosa yugal derecha.
Como antecedente médico personal surge la presencia de ocasionales sangrados intestinales.
Entre los antecedentes familiares se destaca: padre fallecido por tumor intestinal, hermano operado de pólipos intestinales y hermana histerectomizada recientemente con ooforectomía bilateral; la familia desconoce el motivo de esta intervención.
El paciente tiene tres hijos varones sanos.
Al examen clínico se observa circunferencia craneana grande, hipertelorismo, puente nasal ancho, frente prominente y fascies adenoidea. Alrededor de los orificios faciales se observan múltiples lesiones papulares asintomáticas del color de la piel. El examen clínico cuidadoso del resto de la piel revela la presencia de queratosis palmoplantales, que simulan verrugas virales planas y lesiones papulares no queratósicas en axilas, ingles y tórax.
El examen bucal muestra: maxilar inferior desdentado parcial, maxilar superior desdentado total (portador de prótesis completa superior), paladar profundo y lengua fisurada. La mucosa está tapizada por lesiones papulares asintomáticas, redondeadas de bordes definidos, sin cambios de color, superficie lisa, sesiles o pediculadas y de tamaño variable (no más de 1 cm.). Se localizan en mejillas, rebordes, encía adherida, labios y lengua.
La lesión motivo de consulta está en la mejilla en la línea oclusal; es la de mayor tamaño y sufre traumatismo por la prótesis. Su biopsia exéresis muestra una lesión inespecífica de tipo papilomatosa.
Con la presunción de una enfermedad de Cowden el paciente es derivado a un equipo multidisciplinario integrado por genetista, dermatólogo, gastroenterólogo e internista.
Se realiza tomografía computarizada de cráneo (TC) que no muestra alteraciones y endoscopía intestinal que evidencia pólipos en manto. El informe anatomopatológico indica pólipos hiperplásicos, uno de tipo aserrado que asocia cambios displásicos.
Los exámenes paraclínicos y las características fenotípicas del paciente, determinaron que el equipo de genetistas confirmara el diagnóstico de enfermedad de Cowden. Hasta la fecha no se han realizado estudios del material genético.
Posteriormente se citan los hijos para ser examinados. El hijo mediano (21 años) tiene alguna de las características clínicas de la enfermedad (ver tabla 1). El hijo de 27 años expresa débilmente el fenotipo del síndrome, mientras que el hijo menor de 13 años hasta el momento no manifiesta alteraciones.
Remitido por: Dra. Verónica Blanco. E-mail: vblanco@odon.edu.uy""",
"""Nombre: Marina .
Apellidos: Casas Gómez.
NHC: 8455635.
Domicilio: Calle del Dr. Esquerdo 99, Bajo A .
Localidad/ Provincia: Madrid.
CP: 28007.
Datos asistenciales.
Fecha de nacimiento: 21/07/2007.
País: España.
Edad: 7 años Sexo: M.
Fecha de Ingreso: 03/11/2014.
Servicio/ Unidad: Infectología.
Episodio: 524652564.
Médico: Miguel Peñafiel Freire  NºCol: 28 28 97102.
Historia Actual: Niña de siete años nacida en España y sin patología previa que es remitida a la Unidad de Infectología Pediátrica por adenopatías dolorosas submandibulares izquierdas de un mes de evolución, sin respuesta al tratamiento con amoxicilina-ácido clavulánico y antiinflamatorio oral.
La paciente no había presentado fiebre, sudoración nocturna ni pérdida de peso. No estaba vacunada de BCG. Durante un mes al año visita Marruecos, donde tiene contacto con animales (ovejas, corderos) y consume lácteos sin pasteurizar.
Exploración física: En la exploración física presenta un conglomerado submandibular izquierdo de adenopatías, doloroso a la palpación, duro, no rodadero ni adherido, la mayor de las adenopatías de 2 × 2 cm. El resto de la exploración física y anamnesis por aparatos no presentaba hallazgos patológicos.
Resumen de pruebas complementarias: Dada la falta respuesta al tratamiento y teniendo en cuenta los datos epidemiológicos, se realizó la prueba de tuberculina, que fue positiva, con una induración de 10 mm con patrón vesicular a las 72 horas. La radiografía de tórax y la ecografía abdominal fueron normales.
Las serologías de Brucella, virus de Epstein-Barr (VEB) y citomegalovirus (CMV) fueron negativas. La analítica mostró hemograma normal, velocidad de sedimentación globular (VSG) de 47 mm/hora y el interferon-gamma release assays (IGRA) (QuantiFERON(r)) fue positivo, con un valor de 10,15 (rango normal: 0-0,35).
Se realizó una punción aspiración con aguja fina (PAAF) de la adenopatía, que mostraba un material espeso de coloración blanquecina. El cultivo y el estudio citológico revelaron un patrón de linfadenitis granulomatosa necrotizante.
Evolución y comentarios: Dada la escasa afectación de la paciente y la sospecha de adenitis tuberculosa, se pautó un tratamiento domiciliario inicial con isoniacida, rifampicina y pirazinamida, a la espera del cultivo de la muestra obtenida mediante PAAF. El cultivo fue positivo para M. bovis, presentando un antibiograma con resistencia a pirazinamida y sensibilidad a estreptomicina, etambutol, isoniacida y rifampicina. Por este motivo se sustituyó la pirazinamida por etambutol durante dos meses, continuando posteriormente con isoniacida y rifampicina hasta finalizar seis meses. Los cultivos de micobacterias en jugos gástricos fueron negativos. Durante ese tiempo la paciente acudió a revisiones periódicas en el Servicio de Infectología Pediátrica, en las que se constató una buena evolución clínica y la resolución completa del proceso infeccioso.
Remitido por:  Dr. Miguel Peñafiel Freire. Hospital General Universitario Gregorio Marañón. Calle del Dr. Esquerdo, 46, 28007 Madrid. Correo electrónico: mpfreire.89@gmail.com
"""]

text_italian = ["""PZ: Giancarlo Binaghi
CODICE FISCALE: MVANSK92F09W408A
INDIRIZZO: Viale Burcardo 7
CITTÀ : Napoli
CODICE POSTALE: 80139
DATA DI NASCITA: 03/03/1946
ETÀ: 70 anni 
SESSO: M
EMAIL: gcbinaghi@tim.it
DATA DI AMMISSIONE: 12/12/2016
DOTTORE: Eva Viviani
RAPPORTO CLINICO: 70 anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.
È stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.
L'esame fisico ha mostrato buone condizioni generali, con addome e genitali normali; l'esame digitale rettale era coerente con un adenoma prostatico di grado I/IV.
L'analisi delle urine ha mostrato 4 globuli rossi/campo e 0-5 leucociti/campo; il resto del sedimento era normale.
L'emocromo è normale; la biochimica ha mostrato una glicemia di 169 mg/dl e trigliceridi 456 mg/dl; la funzione epatica e renale sono normali. PSA di 1,16 ng/ml.

INDIRIZZATO A: Dott. Bruno Ferrabosco - ASL Napoli 1 Centro, Dipartimento di Endocrinologia e Nutrizione - Stretto Scamarcio 320, 80138 Napoli
EMAIL: bferrabosco@poste.it
""",
"""PZ: Giustino Adinolfi
CODICE FISCALE: DHQFOH39E45C604H
INDIRIZZO: Canale Seddio 62
CITTÀ: Palermo
CODICE POSTALE: 90128
DATA DI NASCITA: 15/07/1979
ETÀ: 38 anni 
SESSO: M
DATA DI AMMISSIONE: 04/03/2014
DOTTORE: Natalia Tamborini
RAPPORTO CLINICO: Storia attuale: paziente maschio di 38 anni, valutato in consultazione per una coxalgia destra di un anno e mezzo di evoluzione, di caratteristiche meccaniche. Ha riportato dolore con l'esercizio fisico e una leggera limitazione della mobilità in certe posture. Meccanico di professione, nel suo tempo libero andava in bicicletta (40 km al giorno nei fine settimana), correva 3 volte alla settimana per 50 minuti e camminava per un'ora nei giorni in cui non correva. Il dolore è aumentato progressivamente e attualmente cammina soltanto.
ESAME FISICO: L'esame fisico ha rivelato un dolore alla palpazione nel terzo medio della regione inguinale destra con dolore alla flessione, all'adduzione e alla rotazione, specialmente alla rotazione interna (manovra di shock positivo). L'equilibrio dell'articolazione dell'anca è di 110o di flessione bilaterale, rotazione interna di 10o destra, 40o sinistra e rotazione esterna di 30o destra, 40o sinistra. L'equilibrio muscolare sulla scala Daniels è 5/5 globale bilaterale. Non c'è dolore alla palpazione dei punti sciatici e le manovre Valleix, Fabere, Lasegue e Bragard sono negative. Non c'era nemmeno dolore alla palpazione della banda iliotibiale e il test Ober era negativo. Nell'esame dell'andatura, abbiamo osservato un'assenza di claudicazione e l'appoggio monopodalico alternato destra-sinistra è stabile, con Trendelenburg negativo. Il paziente aveva dolore quando camminava con una flessione forzata dell'anca e del ginocchio.
Esami complementari: Una semplice radiologia anteroposteriore (AP) ha mostrato una deformità a forma di gobba nella regione femorale cervico-cefalica destra. L'emocromo richiesto, la biochimica, gli ormoni tiroidei, gli anticorpi e i reagenti di fase acuta erano normali.
Alla luce di questi risultati, è stata richiesta una risonanza magnetica dell'anca, che ha mostrato una leggera alterazione subcondrale, con edema osseo nell'anca destra, con irregolarità corticale e leggera sinovite nell'interlinea articolare, con distensione della borsa ilio-psoas. La testa femorale ha un piccolo isolotto osseo e l'angolo alfa è maggiore di 50o. L'artrografia mostra un pizzicamento dell'interlinea articolare antero-superiore e postero-inferiore, un colletto osteofita femorale incipiente con piccole lesioni subcondrali antero-superiore della testa femorale e piccoli focolai di edema nell'acetabolo anterosuperiore. Non c'è necrosi avascolare o lesioni nel labrum. Una lesione cistica juxta-articolare è anche visualizzata in relazione al grande trocantere.
Evoluzione: Con la diagnosi di sindrome da impingement femoroacetabolare di tipo LEVA (nota anche come CAM), al paziente è stato prescritto un trattamento di magnetoterapia (15 sedute) per migliorare i sintomi del dolore e il trofismo osseo.
Il paziente è stato indirizzato all'unità di artroscopia del servizio di traumatologia per la valutazione della chirurgia artroscopica.
Diagnosi principale: sindrome da impingement femoroacetabolare di tipo LEVA.
TRATTAMENTO: Il trattamento farmacologico è prescritto con un'associazione di glucosamina-condroitina solfato orale 400 mg ogni 12 ore per 3 mesi e farmaci antinfiammatori non steroidei al bisogno se il dolore è presente. Viene istruita a riposare negli sport per evitare lo shock all'articolazione, con un'attività fisica attiva controllata, raccomandando di non superare i 90 gradi di flessione dell'anca ed evitando le rotazioni. Prenda un appuntamento per un controllo una volta terminati i solfati di glucosamina-condroitina per vedere il miglioramento clinico. Chiami il numero di assistenza e appuntamenti precedenti dell'ospedale 914 90 66 66 60.
RIFERITO DA: Dott. Paloma Trupiano. Azienda Ospedaliera San Giovanni Addolorata, Strada Damiano 68, 90128 Palermo. Telefono: +39 3664066420. E-mail: p_trupiano@alice.it 
""",
"""PZ: Lina Quasimodo
CODICE FISCALE: DHQFOH39E45C604H
INDIRIZZO: Viale Bianca 95
CITTÀ: Medesano
CODICE POSTALE: 43014 
DATA DI NASCITA: 05/10/1976
ETÀ: 44 anni 
SESSO: Femminile
DATA DI AMMISSIONE: 04/03/2014
DOTTORE: Lina Antonucci
RAPPORTO CLINICO: Donna di 44 anni, si è rivolta alla nostra unità a causa di un aumento di volume nella regione zigomatica e della progressiva limitazione dell'apertura della bocca di 10 mesi di evoluzione. L'anamnesi medica non era rilevante. 
L'esame fisico extraorale ha mostrato un'asimmetria facciale dovuta a un aumento di volume nella guancia sinistra, di limiti diffusi, di consistenza ossea, indolore, con pelle di aspetto normale. L'apertura della bocca era di 30 mm. L'articolazione temporomandibolare (ATM) era normale alla palpazione, senza rumori o dolore all'apertura. All'ortopantomografia abbiamo osservato un processo coronoideo sinistro allargato. L'ipotesi diagnostica era un tumore coronoide.
Prove complementari ed evoluzione: Un'incisione è stata fatta sul bordo anteriore del ramus mandibolare, disinserendo tutte le inserzioni del muscolo temporale. Una pseudocapsula fibrosa è stata trovata intorno all'allargamento, che è stata rilasciata. È stata eseguita una coronoidectomia. L'apertura buccale è stata immediatamente ripristinata a 43 mm. Il decorso postoperatorio è stato poco movimentato e il paziente è stato dimesso dopo 48 ore.
Lo studio istopatologico ha riportato la presenza di fibre, neoformazione ossea e tessuto cartilagineo ialino. La diagnosi era osteocondroma. I controlli successivi sono stati senza problemi. La dimissione definitiva è stata data a 10 mesi, con un rimodellamento quasi completo della deformità della guancia.

RIFERITO DA: Dott. Annunziata Luna. AZIENDA OSPEDALIERA G. BROTZU, Vicolo Margherita 6, 85057 Tramutola. Telefono: 018718869. E-mail: luna@libero.it 
""",
"""PZ: Giancarlo Capuana
CODICE FISCALE: DAJVBV53L03E772A
INDIRIZZO: Canale Faranda 12 Appartamento 89
CITTÀ : Villanova Strisaili
CODICE POSTALE: 08049
DATA DI NASCITA: 10/11/1950
ETÀ: 52 anni 
SESSO: M
EMAIL: gcpna@poste.it
DATA DI AMMISSIONE: 11/11/2002
DOTTORE: Renata Vercelloni-Panicucci
RAPPORTO CLINICO: Un uomo di 52 anni ha consultato il reparto di odontoiatria per un dolore causato da una lesione esofitica sulla mucosa jugale destra.
La sua storia medica personale includeva la presenza di occasionali emorragie intestinali.
L'anamnesi familiare comprende: padre deceduto a causa di un tumore intestinale, fratello operato per polipi intestinali e sorella recentemente isterectomizzata con ooforectomia bilaterale; la famiglia non è a conoscenza del motivo di questa operazione.
Il paziente ha tre figli sani.
L'esame clinico rivela una grande circonferenza della testa, ipertelorismo, ampio ponte nasale, fronte prominente e fascia adenoidea. Si osservano molteplici lesioni papulose asintomatiche color pelle intorno agli orifizi del viso. Un attento esame clinico del resto della pelle rivela cheratosi palmoplantari che simulano verruche virali piatte e lesioni papulari non cheratotiche nelle ascelle, nell'inguine e nel petto.
L'esame orale mostra: mascella inferiore parzialmente edentula, mascella superiore totalmente edentula (con protesi superiore completa), palato profondo e lingua fissurata. La mucosa è coperta da lesioni papulose asintomatiche, arrotondate, con bordi distinti, senza cambiamenti di colore, superficie liscia, sessili o peduncolate e di dimensioni variabili (non più di 1 cm). Si trovano sulle guance, sui bordi, sulle gengive attaccate, sulle labbra e sulla lingua.
La lesione che è il motivo della consultazione è sulla guancia nella linea occlusale; è la più grande e subisce il trauma dovuto alla protesi. La sua biopsia escissa ha mostrato una lesione papillomatosa non specifica.
Con la presunzione della malattia di Cowden, il paziente è stato indirizzato a un team multidisciplinare composto da un genetista, un dermatologo, un gastroenterologo e un internista.
È stata eseguita una tomografia computerizzata (CT) cranica, che non ha mostrato alterazioni, e l'endoscopia intestinale ha rivelato polipi mantellari. Il rapporto anatomopatologico ha indicato polipi iperplastici, uno del tipo seghettato associato a cambiamenti displastici.
Gli esami paraclinici e le caratteristiche fenotipiche del paziente hanno portato il team di genetisti a confermare la diagnosi di malattia di Cowden. Fino ad oggi, non sono stati condotti studi sul materiale genetico.
Successivamente, i bambini sono programmati per i test. Il figlio di mezzo (21 anni) ha alcune delle caratteristiche cliniche della malattia (vedi tabella 1). Il figlio di 27 anni ha una debole espressione del fenotipo della sindrome, mentre il figlio più giovane (13 anni) non ha finora alcuna alterazione.

INDIRIZZATO A: Dott. Mirco Taccola - Casa Di Cura San Luca - Borgo Tiziano 78, 84042 Acerno
EMAIL: mircotcc@fastwebnet.it
""",
"""PZ: Annamaria Dallara
CODICE FISCALE: JDOYKK77M41E977K
INDIRIZZO: Piazza Santi 916
CITTÀ : Moirano
CODICE POSTALE: 15011
DATA DI NASCITA: 23/05/2008
ETÀ: 7 anni 
SESSO: F
EMAIL: N/A
DATA DI AMMISSIONE: 25/07/2015
DOTTORE: Aldo Angiolello
RAPPORTO CLINICO: Una bambina di sette anni nata in Francia senza alcuna patologia precedente è stata indirizzata all'Unità di Malattie Infettive Pediatriche per adenopatie sottomandibolari sinistre dolorose della durata di un mese, senza risposta al trattamento con amoxicillina-acido clavulanico e farmaci antinfiammatori orali.
Il paziente non aveva febbre, sudorazione notturna o perdita di peso. Non è stata vaccinata con il BCG. Per un mese all'anno visita il Marocco, dove ha contatto con gli animali (pecore, agnelli) e consuma prodotti caseari non pastorizzati.
Esame fisico: L'esame fisico ha rivelato un conglomerato sottomandibolare sinistro di linfoadenopatie, doloroso alla palpazione, duro, non rotante e non aderente, il linfonodo più grande misura 2 × 2 cm. Il resto dell'esame fisico e l'anamnesi per apparato non hanno mostrato alcun reperto patologico.
Riassunto dei test complementari: Data la mancanza di risposta al trattamento e tenendo conto dei dati epidemiologici, è stato eseguito un test alla tubercolina, che è risultato positivo, con un'indurazione di 10 mm con pattern vescicolare a 72 ore. La radiografia del torace e l'ecografia addominale erano normali.
Le sierologie per Brucella, Epstein-Barr virus (EBV) e citomegalovirus (CMV) erano negative. L'emocromo ha mostrato un emogramma normale, una velocità di eritrosedimentazione (VES) di 47 mm/ora e il test di rilascio di interferone-gamma (IGRA) (QuantiFERON(r)) era positivo con un valore di 10,15 (range normale: 0-0,35).
Una puntura di aspirazione con ago sottile (FNA) dell'adenopatia ha mostrato un materiale denso e biancastro. La coltura e la citologia hanno rivelato un modello di linfoadenite granulomatosa necrotizzante.
INDIRIZZATO A: Dott. Fabio Zichichi - Ospedale Privato Accreditato Villa Regina - Strada Silvio 95, 15011 Moirano
EMAIL: zichichi@fastwebnet.it
"""]

text_french = ["""PRENOM : Jean
NOM : Dubois
NUMÉRO DE SÉCURITÉ SOCIALE : 1780160471058
ADRESSE : 18 Avenue Matabiau
VILLE : Grenoble
CODE POSTAL : 38000
DATE DE NAISSANCE : 03/03/1946
Âge : 70 ans 
Sexe : H
COURRIEL : jdubois@hotmail.fr
DATE D'ADMISSION : 12/12/2016
MÉDÉCIN : Dr Michel Renaud
RAPPORT CLINIQUE : 70 ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour.
Il nous a été adressé car il présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale.
L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux normaux ; le toucher rectal était compatible avec un adénome de la prostate de grade I/IV.
L'analyse d'urine a montré 4 globules rouges/champ et 0-5 leucocytes/champ ; le reste du sédiment était normal.
Hémogramme normal ; la biochimie a montré une glycémie de 169 mg/dl et des triglycérides de 456 mg/dl ; les fonctions hépatiques et rénales étaient normales. PSA de 1,16 ng/ml.
ADDRESSÉ À : Dre Marie Breton - Centre Hospitalier de Bellevue Service D'Endocrinologie et de Nutrition - Rue Paulin Bussières, 38000 Grenoble
COURRIEL : mariebreton@chb.fr
""",
"""PRENOM : Éric
NOM : Lejeune
NUMÉRO DE SÉCURITÉ SOCIALE : 2730235238020
ADRESSE : 74 Boulevard Riou
VILLE : Sainte Annenec
CODE POSTAL : 49722
DATE DE NAISSANCE : 10/06/1980
Âge : 38 ans 
Sexe : H
COURRIEL : elej@free.fr
DATE D'ADMISSION : 08/11/2018
MÉDÉCIN : Dre Martine Da Costa
RAPPORT CLINIQUE : 
Antécédents : patient de 38 ans, évalué en consultation pour une coxalgie droite d'un an et demi d'évolution, de caractéristiques mécaniques. Il a signalé une douleur à l'exercice et une légère limitation de la mobilité dans certaines postures. Mécanicien de profession, pendant son temps libre, il faisait du vélo (40 km par jour le week-end), courait 3 fois par semaine pendant 50 minutes et marchait pendant une heure les jours où il ne courait pas. La douleur a progressivement augmenté et elle ne fait actuellement que marcher.
EXAMEN PHYSIQUE : L'examen physique a révélé une douleur à la palpation dans le tiers moyen de la région inguinale droite avec une douleur à la flexion, à l'adduction et à la rotation, surtout à la rotation interne (manœuvre de choc positif). L'équilibre de l'articulation de la hanche est de 110o de flexion bilatérale, 10o de rotation interne droite, 40o de rotation interne gauche et 30o de rotation externe droite, 40o de rotation externe gauche. L'équilibre musculaire sur l'échelle de Daniels est 5/5 global bilatéral. Pas de douleur à la palpation des points sciatiques et les manœuvres de Valleix, Fabere, Lasegue et Bragard sont négatives. Pas non plus de douleur à la palpation de la bandelette iliotibiale et le test d'Ober était négatif. A l'examen de la marche, nous avons observé une absence de claudication et la posture monopodale alternée droite-gauche était stable, avec un Trendelenburg négatif. Le patient avait des douleurs lors de la marche avec une flexion forcée de la hanche et du genou.
Examens complémentaires : Une radiologie ordinaire antéropostérieure (AP) a montré une déformation en forme de bosse dans la région fémorale cervicale droite. L'hémogramme, la biochimie, les hormones thyroïdiennes, les anticorps et les réactifs de phase aiguë requis étaient normaux.
Au vu de ces résultats, une IRM de la hanche a été demandée, qui a montré de légères modifications sous-chondrales, avec un œdème osseux dans la hanche droite, avec une irrégularité corticale et une légère synovite dans l'interligne articulaire, avec une distension de la bourse ilio-psoas. La tête fémorale présente un petit îlot osseux et l'angle alpha est supérieur à 50o. L'arthrographie montre un pincement de l'interligne articulaire antéro-supérieur et postéro-inférieur, un col fémoral ostéophytique naissant avec de petites lésions sous-chondrales antéro-supérieures de la tête fémorale et de petits foyers d'œdème dans l'acétabulum antéro-supérieur. Pas de nécrose avasculaire ni de lésions du labrum. Une lésion kystique juxta-articulaire est également observée en regard du grand trochanter.
Evolution : Avec le diagnostic de syndrome de conflit fémoroacétabulaire de type LEVA (également connu sous le nom de CAM), le patient s'est vu prescrire un traitement par magnétothérapie (15 séances) pour améliorer les symptômes de la douleur et le trophisme osseux.
Le patient a été adressé à l'unité d'arthroscopie du service de traumatologie pour une évaluation de la chirurgie arthroscopique.
Diagnostic principal : syndrome du conflit fémoroacétabulaire de type LEVA.
TRAITEMENT : Un traitement pharmacologique est prescrit avec une combinaison de glucosamine-chondroïtine sulfate par voie orale 400 mg toutes les 12 heures pendant 3 mois et des anti-inflammatoires non stéroïdiens si nécessaire en cas de douleur. Elle a pour consigne de se reposer dans les sports pour éviter tout choc à l'articulation, avec une activité active contrôlée, en recommandant de ne pas dépasser 90 degrés de flexion de la hanche et d'éviter la rotation. Prenez rendez-vous pour un suivi une fois les sulfates de glucosamine-chondroïtine terminés afin de constater une amélioration clinique. Appelez le numéro des soins et des rendez-vous précédents de l'hôpital : 0470892301.
ADDRESSÉ À : Pr Diane Martinez - Centre Hospitalier Saint Louis - 30 Rue Garcia, 49000 Angers
COURRIEL : dmartinez@chsl.fr
""",
"""PRENOM : Aurélie
NOM : Perrier
NUMÉRO DE SÉCURITÉ SOCIALE : 2190150502008
ADRESSE : 8 Rue Faivre
VILLE : Bousquet
CODE POSTAL : 11631
DATE DE NAISSANCE : 22/10/1960
Âge : 44 ans 
Sexe : Femme
COURRIEL : aurelieprr@orange.fr
DATE D'ADMISSION : 31/12/2004
MÉDÉCIN : Dr Alain Bouvet
RAPPORT CLINIQUE : 
Motif de l'admission : Femme âgée de 44 ans, adressée à notre unité en raison d'une augmentation de volume dans la région zygomatique et d'une limitation progressive de l'ouverture de la bouche depuis 10 mois. Les antécédents médicaux n'étaient pas pertinents. 
Examen : L'examen physique extraoral a montré une asymétrie faciale due à une augmentation de volume de la joue gauche, de limites diffuses, de consistance osseuse, indolore, avec une peau d'apparence normale. L'ouverture de la bouche était de 30 mm. L'articulation temporomandibulaire (ATM) était normale à la palpation, avec des bruits ou des douleurs à l'ouverture. Sur l'orthopantomographie, nous avons observé une hypertrophie du processus coronoïde gauche. L'hypothèse diagnostique était une tumeur coronoïde.
Tests complémentaires et évolution : Une incision a été pratiquée sur le bord antérieur du ramus mandibulaire, en désinsérant toutes les insertions du muscle temporal. Une pseudocapsule fibreuse a été trouvée autour de l'élargissement, qui a été libérée. Une coronoïdectomie a été effectuée. L'ouverture buccale a été immédiatement restaurée à 43 mm. L'évolution postopératoire s'est déroulée sans incident avec une sortie de l'hôpital après 48 heures.
L'étude histopathologique a signalé la présence de fibres, d'os de néoformation et de tissu cartilagineux hyalin. Les contrôles ultérieurs se sont déroulés sans incident. La sortie définitive a été donnée à 10 mois, avec un remodelage presque complet de la déformation des joues.
ADDRESSÉ À : Dr Joseph Pasquier - Centre Hospitalier de Carcassonne - 1060 Chemin de la Madeleine, 11000 Carcassonne
COURRIEL : pasquier@orange.fr
""",
"""PRENOM : Emmanuel
NOM : Pétion
NUMÉRO DE SÉCURITÉ SOCIALE : 1580786194210
ADRESSE : 162 Chemin Evrard
VILLE : Bodin
CODE POSTAL : 52599
DATE DE NAISSANCE : 08/02/1948
Âge : 52 ans 
Sexe : H
COURRIEL : petion@free.fr
DATE D'ADMISSION : 19/06/2000
MÉDÉCIN : Dre Alice Bertrand
RAPPORT CLINIQUE : Un homme de 52 ans, originaire de Haïti, a consulté le service de dentisterie pour une douleur causée par une lésion exophytique sur la muqueuse jugale droite.
Ses antécédents médicaux personnels comprenaient la présence de saignements intestinaux occasionnels.
Les antécédents familiaux comprennent : le père est décédé en raison d'une tumeur intestinale, le frère a été opéré pour des polypes intestinaux et la sœur a récemment subi une hystérectomie avec ovariectomie bilatérale ; la famille ignore la raison de cette opération.
L'homme a trois fils en bonne santé.
L'examen clinique révèle un grand tour de tête, un hypertélorisme, une large arête nasale, un front proéminent et un fascia adénoïde. De multiples lésions papuleuses asymptomatiques de couleur chair sont observées autour des orifices du visage. Un examen clinique attentif du reste de la peau révèle des kératoses palmoplantaires simulant des verrues virales plates et des lésions papulaires non kératosiques dans les aisselles, l'aine et la poitrine.
L'examen oral montre : un édenté partiel de la mâchoire inférieure, un édenté total de la mâchoire supérieure (avec une prothèse supérieure complète), un palais profond et une langue fissurée. La muqueuse est couverte de lésions papuleuses asymptomatiques, arrondies, aux bords distincts, sans changement de couleur, à surface lisse, sessiles ou pédonculées et de taille variable (pas plus de 1 cm). Ils sont situés sur les joues, les rebords, les gencives attachées, les lèvres et la langue.
Avec la présomption de la maladie de Cowden, le patient a été adressé à une équipe multidisciplinaire.
Une tomodensitométrie (TDM) crânienne a été réalisée, qui n'a montré aucune altération, et l'endoscopie intestinale a révélé des polypes mantelliques. Le rapport anatomopathologique indiquait des polypes hyperplasiques, dont un de type dentelé associé à des changements dysplasiques.
Les examens paracliniques et les caractéristiques phénotypiques du patient ont conduit l'équipe de généticiens à confirmer le diagnostic de la maladie de Cowden. A ce jour, aucune étude du matériel génétique n'a été réalisée.
ADDRESSÉ À : Dr Marc Lefort - Centre Hospitalier Spécialisé Esquirol - Impasse Hortense Briand, 73168 Montgilbert
COURRIEL : marclefort@centrehospit.fr
""",
"""PRENOM : Clothilde
NOM : Gilbert
NUMÉRO DE SÉCURITÉ SOCIALE : 1961051108178
ADRESSE : 38 Boulevard de Weiss
VILLE : Tournay
CODE POSTAL : 65079
DATE DE NAISSANCE : 01/01/2012
Âge : 7 ans 
Sexe : F
COURRIEL : N/A
DATE D'ADMISSION : 07/07/2019
MÉDÉCIN : Dr Luc Legendre
RAPPORT CLINIQUE : Une jeune fille de 7 ans née en Espagne, sans pathologie antérieure, a été adressée à l'unité de pédiatrie des maladies infectieuses pour des adénopathies submandibulaires gauches douloureuses depuis un mois, sans réponse au traitement par amoxicilline-acide clavulanique et anti-inflammatoires oraux.
Pas de fièvre, ni de sueurs nocturnes, ni de perte de poids. Elle n'a pas été vaccinée avec le BCG. Pendant un mois par an, elle se rend au Maroc, où elle est en contact avec des animaux (moutons, agneaux) et consomme des produits laitiers non pasteurisés.
Examen physique : L'examen physique a révélé un conglomérat d'adénopathies sous-mandibulaire gauche, douloureux à la palpation, dur, non roulant et non adhérent, le plus gros ganglion lymphatique mesurant 2 × 2 cm. Le reste de l'examen physique et l'anamnèse par appareil n'ont montré aucun résultat pathologique.
Résumé des examens complémentaires : Vu l'absence de réponse au traitement et compte tenu des données épidémiologiques, un test tuberculinique a été réalisé, qui s'est révélé positif, avec une induration de 10 mm avec un aspect vésiculaire à 72 heures. La radiographie du thorax et l'échographie abdominale étaient normales.
Les sérologies de Brucella, du virus d'Epstein-Barr (EBV) et du cytomégalovirus (CMV) étaient négatives. La NFS a montré un hémogramme normal, une vitesse de sédimentation érythrocytaire (VS) de 47 mm/heure et un test de libération d'interféron-gamma (IGRA) (QuantiFERON(r)) positif avec une valeur de 10,15 (plage normale : 0-0,35).
Une ponction à l'aiguille fine (FNA) de l'adénopathie a montré une matière épaisse et blanchâtre. La culture et la cytologie ont révélé un modèle de lymphadénite granulomateuse nécrosante.
Evolution et commentaires : Compte tenu de la faible implication du patient et de la suspicion d'adénite tuberculeuse, un traitement initial à domicile par isoniazide, rifampicine et pyrazinamide a été prescrit, en attendant la culture du prélèvement obtenu par FNA. La culture était positive, avec un antibiogramme montrant une résistance à la pyrazinamide et une sensibilité à la streptomycine, l'éthambutol, l'isoniazide et la rifampicine. Pour cette raison, le pyrazinamide a été remplacé par l'éthambutol pendant deux d'entre eux, puis par l'isoniazide et la rifampicine jusqu'à la fin des six mois. Les cultures mycobactériennes dans le suc gastrique étaient négatives. Pendant cette période, le patient a subi des contrôles réguliers au service des maladies infectieuses pédiatriques, où une bonne évolution clinique et une résolution complète du processus infectieux ont été observées.
ADDRESSÉ À : Pr Michelle Charrier - Centre Hospitalier De Bigorre - Boulevard de Lattre de Tassigny, 65013 Tarbes
COURRIEL : charrier@chb.fr
"""]

## **3. Define Spark NLP pipeline**

In [8]:
lang_dict = {"en" : text_english,
             "de" : text_german,
             "es" : text_spanish,
             "it" : text_italian,
             "fr" : text_french
             }

In [9]:
for  lang, text in lang_dict.items():
    if lang == "es":
        deid_pipeline = PretrainedPipeline("clinical_deidentification_augmented", "es", "clinical/models")
        deid_res = deid_pipeline.annotate(text)[0]
        df= pd.DataFrame(list(zip(deid_res["sentence"], deid_res["masked"], deid_res["masked_with_chars"], deid_res["masked_fixed_length_chars"], deid_res["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

        print(df.head())
    else:
        deid_pipeline = PretrainedPipeline("clinical_deidentification", lang, "clinical/models")
        deid_res = deid_pipeline.annotate(text)[0]
        df= pd.DataFrame(list(zip(deid_res["sentence"], deid_res["masked"],
                          deid_res["masked_with_chars"], deid_res["masked_fixed_length_chars"], deid_res["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

        print(df.head()) 
    

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]
                                                                                     Sentence                                                                                      Masked                                                                           Masked with Chars                                                                     Masked with Fixed Chars                                                                                  Obfuscated
0                                        Record date: 2069-04-07\nMr. Villegas is seen today.                                           Record date: <DATE>\nMr. <PATIENT> is seen today.                                        Record date: [********]\nMr. [******] is seen today.                                                  Record date: ****\nMr. **** is seen today.                                     Record date: 2069-04-28\nMr